## Scenario 1, Simple Http Server
### Making the dataset, thats is a load on a pod named "httpdia".
prerequisites:  

minikube start

https://httpd.apache.org/docs/2.4/programs/ab.html

In [1]:
import time
from otumba.utils import manage_pods
from otumba.utils import get_pod_metrics_scenario_1
from otumba.GeneralPod import Pod

In [2]:
ServerPrometheus = "http://192.168.49.2:30000/"
numberpods = 10
request = 1000000
concurrency = 14000
host = "http://httpdia.default.svc.cluster.local/"
filedataset = "dataset.csv"
filedataset = "scenario01-"+str(request)+"-"+str(concurrency)+".csv"
requestpod = "load-generator-jupyter-c"
requestedpod = "httpdia"

### filedataset

In [3]:
lapso="5m"
namespace="default"

In [4]:
respuesta= get_pod_metrics_scenario_1 (server = ServerPrometheus, lapso = lapso, 
                                       namespace = namespace, podname = requestedpod, 
                                        requestpodname = requestpod)

In [5]:
from datetime import datetime
import csv
respuesta["load"] = '1000'
respuesta["date"] = datetime.now()
header = list(respuesta.keys())
print(respuesta)

{'pods': '10.0', 'file_descriptors': '344.4', 'receive_packets': '17323.1', 'transmit_packets': '13457.3', 'dropped_packets': '230.3', 'cpu_usage_seconds': '1.4200044228', 'cpu_throttled_seconds': '0.2190041543', 'memory_working_bytes': '46395946.2', 'memory_usage_bytes': '20.0', 'load': '1000', 'date': datetime.datetime(2021, 8, 16, 21, 51, 46, 519848)}


In [6]:
f = open(filedataset, "w", newline='')
writer = csv.DictWriter(f, fieldnames = header)
writer.writeheader()

In [7]:
manage_pods(numberpods, "httpd", requestedpod, "500m", "200m", "default")

Pod Exist, updating replicas


In [8]:
loadpod = Pod(namepod = requestpod, 
              dockerimage = "httpd", namespace = "default", shell = "/bin/sh")

In [9]:
loadpod.create()

In [10]:
carga = "ab -n "+str(request)+" -c "+str(concurrency)+" "+host

In [ ]:
loadpod.exec_command(carga)
countstdout=0
while loadpod.resp.is_open():
    resp_stdout=""
    resp_stderr=""
    longitud =0
    respuesta = get_pod_metrics_scenario_1 (server = ServerPrometheus, lapso = lapso, 
                                       namespace = namespace, podname = requestedpod, 
                                        requestpodname = requestpod)
    respuesta["load"] = str(concurrency)
    respuesta["date"] = datetime.now()
    writer.writerow(respuesta)
    if loadpod.resp.peek_stdout():
        resp_stdout= loadpod.resp.read_stdout()
        longitud= len(resp_stdout)
        countstdout= countstdout+1
    if loadpod.resp.peek_stderr():
        resp_stderr=loadpod.resp.read_stderr()
        print("STDERR: %s" % resp_stderr)
        inicioerror = resp_stderr.find("Completed")
        if inicioerror == -1:
            failedrequest=resp_stderr
            break
    if  (longitud >300):
        inicio = resp_stdout.find("Failed requests")
        if (inicio > -1):
            fin = resp_stdout.find("Total transferred")
            failedrequest=resp_stdout[inicio+16:fin-1]
        finalresponse = resp_stdout.split("\n")
    if countstdout==2:
        break
    #loadpod.resp.update(timeout=1)
respuesta =  failedrequest.strip()
f.close()

ab -n 1000000 -c 14000 http://httpdia.default.svc.cluster.local/
STDERR: Completed 100000 requests

STDERR: Completed 200000 requests



In [ ]:
import csv
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
import seaborn as sns

dataframe = pd.read_csv(filedataset)

In [ ]:
plt.figure(figsize=(20,12))
plt.title("cpu_usage_seconds, pods ="+str(numberpods)+" concurrency = "+ str(concurrency) )
plt.xticks(rotation = 90)
plt.plot(dataframe.dropped_packets, c = "cyan", marker = "o")
#plt.plot(dataframe.memory_usage_bytes, c = "green", marker = "o")
plt.show()